In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Concatenate
#from tensorflow.keras.layers import Model
from tensorflow.keras.layers import LSTM
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers.experimental import Adagrad
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate, Lambda
train_b = True

In [ ]:
def RMSE(y_test,y_pred):
    y_test = list(y_test); y_test = [float(e) for e in y_test]
    y_pred = list(y_pred); y_pred = [float(e) for e in y_pred]
    n = len(y_test)
    rmse_list = [float(y_test[i]-y_pred[i])*float(y_test[i]-y_pred[i]) for i in range(n)]
    return np.sqrt(sum(rmse_list)/float(n))
def RMSE_list(y_test,y_pred):
    y_test = list(y_test); 
    y_test = [list(ln) for ln in y_test]
    y_pred = list(y_pred); 
    y_pred = [list(ln) for ln in y_pred]
    n = len(y_test); m = len(y_test[0])
    rmse_list = [np.square(float(y_test[i][j]-y_pred[i][j])) for i in range(n) for j in range(m)]
    return np.sqrt(sum(rmse_list)/float(n))

#Load and Reshape Data

In [ ]:
DATA_DIR = "/content/drive/MyDrive/Capstone_Grid_Load_Forecasting/Data"
hist_window=18;forecast_window=18
#hist_window=int(12*60/5)

In [ ]:
data = []
#for y in range(2017,2023):
for y in [2021]:
  df = pd.read_csv(f"{DATA_DIR}/Total_Data_{y}_5m.csv")
  data.append(df)
data = pd.concat(data)



future_var = ['Global Horizontal UV Irradiance (280-400nm)','HourlyDryBulbTemperature']
for c in future_var:
  data[c+" Future"] = data[c].shift(-forecast_window).backfill()

used_var = ['N.Y.C.','Global Horizontal UV Irradiance (280-400nm) Future','HourlyDryBulbTemperature Future']
data = data[used_var].backfill();
data = data.dropna(how='any')

for c in used_var:
  #print(data[c].head(15));
  print(c,len(data[c].index),data[c].mean(),data[c].std())

print(data.head(100).to_string())
n = len(data.index)
t = int(7*24*(60/5))
num_var = len(used_var)

N.Y.C. 105102 5574.236639466899 1157.7346584901197
Global Horizontal UV Irradiance (280-400nm) Future 105102 9.306859028372438 14.709639920529495
HourlyDryBulbTemperature Future 105102 56.03478681325037 16.505866228760585
       N.Y.C.  Global Horizontal UV Irradiance (280-400nm) Future  HourlyDryBulbTemperature Future
0   4915.3086                                            0.000000                        33.000000
1   4892.8174                                            0.000000                        33.000000
2   4868.1533                                            0.000000                        33.000000
3   4856.5923                                            0.000000                        33.000000
4   4829.3720                                            0.000000                        33.000000
5   4810.1543                                            0.000000                        32.933333
6   4785.9280                                            0.000000                    

In [ ]:
m = 5574.113154123897;std = 1157.6820234795014
m_s = 9.305353909378894 ;std_s = 14.708926535057076
m_t = 56.030910041113;std_t = 16.507268780702415


m_list = [m,m_s,m_t]
std_list = [std,std_s,std_t]

def untransform_dataset_old(dataset,mi=m_list,stdi=std_list):
  s = dataset.shape
  if len(s)==3:
    ns,nv,tw = s #ns = number of samples, nv = number of variables, tw = time window
    dataset=dataset.reshape(nv,ns*tw)
    dataset = np.array([(dataset[i]*stdi[i])+mi[i] for i in range(s[1])])
    dataset=dataset.reshape(s[0],s[1],s[2])
  else: 
    dataset=(dataset*stdi[0])+mi[0]
  return dataset

def create_dataset(dataset, hist_window=18,forecast_window=18):
  #Lookback of 18 includes 90 minutes of previous data
  dataX, dataY = [], []
  for i in range(len(dataset)-hist_window-forecast_window):
    a = dataset[i:(i+hist_window)].reshape(-1); dataX.append(a)
    dataY.append(dataset[i + hist_window:  i + hist_window+forecast_window])
  return np.array(dataX), np.array(dataY)
def create_dataset_mult(dataset, hist_window=18,forecast_window=18,num_var=3):
  #Lookback of 18 includes 90 minutes of previous data
  dataX,dataZ, dataY = [], [], []
  for i in range(dataset.shape[0]-hist_window-forecast_window):
    a = dataset[i:(i+hist_window)].T
    dataX.append(a)
    a = dataset[i:(i+hist_window),0].T
    dataZ.append(a)
    
    e = dataset[i + hist_window:  i + hist_window+forecast_window, 0]
    dataY.append(e)
  return np.array(dataX), np.array(dataZ), np.array(dataY)

def transform_dataset(dataset,mi=m_list,stdi=std_list):
  s = dataset.shape
  if len(s)==2:
    #ns,nv,tw = s #ns = number of samples, nv = number of variables, tw = time window
    ns,nv = s
    dataset=dataset.T 
    dataset = np.array([(dataset[i]-mi[i])/stdi[i] for i in range(s[1])])
    dataset=dataset.T 
  else:
    dataset=(dataset-mi[0])/stdi[0] 
  return dataset
def untransform_dataset(dataset,mi=m_list,stdi=std_list):
  dataset=(dataset*stdi[0])+mi[0]
  return dataset  
dataset = data[used_var].values
dataset_xfr = transform_dataset(dataset)

x,l,y = create_dataset_mult(dataset_xfr,hist_window,forecast_window,num_var)
train,train_l,label_t = x[0:n-t],l[0:n-t],y[0:n-t]
valid,valid_l,label_v = x[n-t:], l[n-t:], y[n-t:]

In [ ]:
# reshape input to be [samples, time steps, features]
#print(train.shape)
#train = np.reshape(train, (train.shape[0], num_var, train.shape[1]))
#valid = np.reshape(valid, (valid.shape[0], num_var, valid.shape[1]))

#Build Model

In [ ]:
for ln in train[0:5]: print(ln[0])

[-0.56907211 -0.5884999  -0.60980463 -0.61979096 -0.64330372 -0.65990388
 -0.68083043 -0.67544122 -0.69008142 -0.7064252  -0.72160026 -0.74121057
 -0.7419454  -0.74958593 -0.80584576 -0.81127852 -0.79814529 -0.80015335]
[-0.5884999  -0.60980463 -0.61979096 -0.64330372 -0.65990388 -0.68083043
 -0.67544122 -0.69008142 -0.7064252  -0.72160026 -0.74121057 -0.7419454
 -0.74958593 -0.80584576 -0.81127852 -0.79814529 -0.80015335 -0.81706128]
[-0.60980463 -0.61979096 -0.64330372 -0.65990388 -0.68083043 -0.67544122
 -0.69008142 -0.7064252  -0.72160026 -0.74121057 -0.7419454  -0.74958593
 -0.80584576 -0.81127852 -0.79814529 -0.80015335 -0.81706128 -0.82611644]
[-0.61979096 -0.64330372 -0.65990388 -0.68083043 -0.67544122 -0.69008142
 -0.7064252  -0.72160026 -0.74121057 -0.7419454  -0.74958593 -0.80584576
 -0.81127852 -0.79814529 -0.80015335 -0.81706128 -0.82611644 -0.8323508 ]
[-0.64330372 -0.65990388 -0.68083043 -0.67544122 -0.69008142 -0.7064252
 -0.72160026 -0.74121057 -0.7419454  -0.74958593 

In [ ]:
import datetime
MODEL_DIR = "/content/drive/MyDrive/Capstone_Grid_Load_Forecasting/Model/Test"
def create_multi_input_model(recurrent_dropout=0.05,lstm_multiple = 3,
                      learning_rate=0.00005,var_num = 4,
                      hist_window=18,forecast_window=18):
  
  load_input = ly.Input(shape=(1,hist_window), name='load_input')
  weather =  ly.Input(shape=(var_num,hist_window), name='weather_input')
  lstm = ly.LSTM(lstm_multiple * hist_window,  recurrent_dropout=recurrent_dropout, name = 'lstm')(load_input)
  lstm = ly.Reshape((lstm_multiple, hist_window))(lstm)
  all_input = ly.Concatenate(axis=1)([lstm, weather])
  forcast = ly.Dense(forecast_window)(all_input)
  forcast = ly.Reshape(((var_num+lstm_multiple)*hist_window,))(forcast) # can use a convolution layer here? 
  forcast = ly.Dense(forecast_window)(forcast)

  model = Model(inputs = [load_input, weather], outputs = [forcast])

  return model

model = create_multi_input_model( var_num=num_var)
model.summary()
plot_model(model, show_shapes=True, dpi=90)

NameError: ignored

# example

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Concatenate
#from tensorflow.keras.layers import Model
from tensorflow.keras.layers import LSTM
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers.experimental import Adagrad
from keras.models import Model
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate
from keras.optimizers import Adagrad

first_input = Input(shape=(2, ))
first_dense = Dense(1, )(first_input)

second_input = Input(shape=(2, ))
second_dense = Dense(1, )(second_input)

merge_one = concatenate([first_dense, second_dense])

third_input = Input(shape=(1, ))
merge_two = concatenate([merge_one, third_input])

model = Model(inputs=[first_input, second_input, third_input], outputs=merge_two)
ada_grad = Adagrad(lr=0.1, epsilon=1e-08, decay=0.0)
model.compile(optimizer=ada_grad, loss='binary_crossentropy',
               metrics=['accuracy'])

plot_model(model, show_shapes=True, dpi=90)

#train

In [ ]:
#Fit Model
model_name = 'test_model_slope_with_solar_weather_future_100.h5'
print(datetime.datetime.now())
load = []
if train_b:
  model.fit([train_l,train], label_t, epochs=5, batch_size=1, verbose=2)
  model.save(f"{MODEL_DIR}/{model_name}",save_format='h5')
else: 
  model.load_weights(f"{MODEL_DIR}/{model_name}") 

In [ ]:
# make predictions
valid,label_v = x[n-t:], y[n-t:]
label_p   = model.predict(valid)

## invert predictions
label_p = untransform_dataset(label_p)
label_v = untransform_dataset(label_v)

# calculate root mean squared error
print('Test Score: %.2f RMSE' % (RMSE_list(label_v, label_p)))

for i,ln in enumerate(label_p):
  if i==0: 
    label_p_plot=list(label_p[0])
    label_v_plot=list(label_v[0])
  else:
    label_p_plot=label_p_plot+ [label_p[i][-1]]
    label_v_plot=label_v_plot+ [label_v[i][-1]]

print('Test Score: %.2f RMSE' % (RMSE(label_v_plot, label_p_plot)))

plt.plot(label_v_plot);
plt.plot(label_p_plot)
plt.show()